In [1]:
#basic package
import json
import tqdm
import os
import numpy as np
import glob
import pandas as pd
import sys
import shutil
import time
import datetime as dt
import itertools
import time
import re
from collections import Counter
from collections import defaultdict
import operator
import math
import functools
import collections

#plot
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
from UTILS import preprocessing_experiment2, general_cleaning, simple_cleaning_experiment2,verification_based_on_initial_record,\
HenVariable, print_color, boxplot_distribution_entropy, heatmap_duration_perzone_perhen, \
time_series_henColumn_tsRow, cleaning_mouvement_records
import config_experiment2 as config

C:\Users\camil\Anaconda3\lib\site-packages\scipy\sparse\sparsetools.py:21: DeprecationWarning: `scipy.sparse.sparsetools` is deprecated!
scipy.sparse.sparsetools is a private module for scipy.sparse, and should not be used.
  _deprecated()


# Define parameters

In [3]:
print('change the configuration file if not done yet!')
path_extracted_data = config.path_extracted_data
id_run = config.id_run
dico_matching = config.dico_matching
print(id_run)
#print('The date that will be removed are: \n %s'%' \n '.join([str(d) for d in sorted(config.li_date2remove)]))

change the configuration file if not done yet!
VFVFVF_


# Preprocessing

path_initial_data = r'R:\VPHI\Welfare\2- Research Projects\Laura C-Klara G\tracking_Data'
#Note that as there is no standardize name (i.e. start with log_), we will simply open all csv files. If there is other csv file
#it wont work
p = glob.glob(os.path.join(path_initial_data,'TagUpdates*_*','*.csv'))
#print('There is %d log files:\n  %s'%(len(p),'  \n  '.join(p)))
path_FocalBird = os.path.join(path_initial_data,'full_tagID_long.xlsx')
df = preprocessing_experiment2(p, path_FocalBird, config)
print(df.shape)
display(df.head(3))
display(df.tail(3))

# Cleaning

### general cleaning (adding ms)

#for being more reproductible, we open the file that was saved from cleaning
df = pd.read_csv(os.path.join(path_extracted_data, id_run+'_PreprocessRecords.csv'), sep=';', parse_dates=['Timestamp'],
                 index_col=0) 
print(df.shape)
df.head(3)

#same for each experiment
df = general_cleaning(df, config)
print(df.shape)
df.tail(3)

### cleaning rules info

Only if needed. Exemple check if system is working better after recalibration

#for being more reproductible, we open the file that was saved from cleaning
df = pd.read_csv(os.path.join(path_extracted_data, id_run+'_records_GeneralCleaning.csv'), sep=';', 
                 parse_dates=['Timestamp', 'Timestamp_initial', 'day']) 
print(df.shape)
df.head(3)

df = cleaning_mouvement_records(df, config, nbr_block_repetition=3, flickering_type1=True)
print(df.shape)
df

### Binning

##### Create one second interval time serie

In [4]:
#for being more reproductible, we open the file that was saved from cleaning
df = pd.read_csv(os.path.join(path_extracted_data, id_run+'_records_GeneralCleaning.csv'), sep=';', 
                 parse_dates=['Timestamp', 'Timestamp_initial', 'date']) 
print(df.shape)
df.tail(3)

(12453168, 10)


,Timestamp,HenID,Zone,PenID,log_file_name,date,ts_order_logname,ts_order_list,ms,Timestamp_initial
12453165,2020-07-18 23:59:56,hen_51,2 Zone,3,2020-07-15_224155_2020-07-19_055005csv,2020-07-18,2020-07-15_224155_2020-07-19_055005csv_98992,2020-07-15_224155_2020-07-19_055005csv_98992,0.0,2020-07-18 23:59:56
12453166,2020-07-18 23:59:59,hen_8,5 Zone,12,2020-07-15_054035_2020-07-19_041254csv,2020-07-18,2020-07-15_054035_2020-07-19_041254csv_84479,2020-07-15_054035_2020-07-19_041254csv_84479,0.0,2020-07-18 23:59:59
12453167,2020-07-18 23:59:59,hen_82,5 Zone,11,2020-07-15_054035_2020-07-19_041254csv,2020-07-18,2020-07-15_054035_2020-07-19_041254csv_84480,2020-07-15_054035_2020-07-19_041254csv_84480,0.0,2020-07-18 23:59:59


START_TIME = time.clock()
df_ts = time_series_henColumn_tsRow(df, config, col_ts='Zone', save=True, hen_time_series=False)
END_TIME = time.clock()
print ("Total running time to create time series: %.2f mn" %((END_TIME-START_TIME)/60)) 
print(df_ts.shape)
df_ts.head(3)
#more efficient: by month-batch

##### binning based on the one second time series

In [5]:
df.head(3)

,Timestamp,HenID,Zone,PenID,log_file_name,date,ts_order_logname,ts_order_list,ms,Timestamp_initial
0,2019-10-22 07:37:52.000000,hen_71,3 Zone,10,2019-10-16_113343_2019-11-02_060257csv,2019-10-22,2019-10-16_113343_2019-11-02_060257csv_292,2019-10-16_113343_2019-11-02_060257csv_292 201...,0.000000,2019-10-22 07:37:52
1,2019-10-22 07:37:52.000000,hen_39,4 Zone,12,2019-10-16_113343_2019-11-02_060257csv,2019-10-22,2019-10-16_113343_2019-11-02_060257csv_293,2019-10-16_113343_2019-11-02_060257csv_293 201...,0.000000,2019-10-22 07:37:52
2,2019-10-22 07:37:52.166666,hen_71,3 Zone,10,2019-10-16_113343_2019-11-02_060257csv,2019-10-22,2019-10-16_113343_2019-11-02_060257csv_294,2019-10-16_113343_2019-11-02_060257csv_292 201...,166666.666667,2019-10-22 07:37:52


In [ ]:
for p in df['PenID'].unique():
    df_final = simple_cleaning_experiment2(df[df['PenID']==p], config, 60)

create time series
in this time series there is 18 hens
The initial starting date in over all is: 2019-10-22 07:37:52, and the ending date will be: 2020-07-18 23:53:19
But note that birds may have different ending and starting date
and after ending the last day at midnight : 2019-10-22 07:37:52, and the ending date will be: 2020-07-18 23:59:59
[17 16 15 14 12 11  9  8  7  6  5  4  3  2  1  0]
-------------- Lets remove timestamp without all hen
as we want the time series to start at the same time, we remove the dates without info on each hen, making us start on  2019-10-22 07:38:43


In [ ]:
#by batch as we want at minute level, then we are sure each day will not need the day before (it needs jsut to know where it is,
#i.e. to create seconds time series but thats it)
def simple_cleaning_experiment2(df, config, nbr_sec_mean, mi=None, ma=None, save=True):
    
    ''' 
    *input: nbr_sec_mean: period, df_ts: time serie dataframe, typically created by the function "time_series_henColumn_tsRow()"
    *output: a csv where timestamp ts results in the bining the all record from ts-period to ts]
    *main idea: create time series for each hen by taking the most frequent zone for each "nbr_sec_mean" seconds period
    *programming main idea: First we create a list of timestamp including only the one we want (i.e. one per nbr_sec_mean seconds). Then we match the old timestamp with the smallest of the list taht is beger of equal to the actual timestamp
    '''
    
    #start recording the time it last
    START_TIME = time.clock()
    
    print('create time series')
    df_ts = time_series_henColumn_tsRow(df, config, col_ts='Zone', save=True, hen_time_series=False)
    print('finish creating time series')
      
    #initialize parameters
    id_run = config.id_run
    path_extracted_data = config.path_extracted_data
    nbr_sec = 1 #should stay one for now
    
    #create a directory if not existing
    path_ = os.path.join(path_extracted_data, 'HensTimeSeries')
    if not os.path.exists(os.path.join(path_)):
        os.makedirs(os.path.join(path_))
        
    #######################################################################################################################
    ##### create a list of dates that we want starting from our initial and end dates with the wanted binning period ######
    if mi==None:
        mi = min(df_ts['Timestamp'].tolist())
    if ma==None:
        ma = max(df_ts['Timestamp'].tolist())
    #keeping dataframe that is linked to these dates
    df_ts = df_ts[(df_ts['Timestamp']>=mi) & (df_ts['Timestamp']<=ma)]
    
    #on arondi a la minute du bas/haut pour ne pas rater des records 
    Daterange = pd.date_range(start = mi-dt.timedelta(seconds=mi.second), 
                              end = ma+dt.timedelta(seconds=60-ma.second), 
                              freq = 'S')    
    print('The starting date of the datetime list is: %s, and the ending date is: %s'%(str(Daterange[0]), str(Daterange[-1])))
    #take only the wanted timestamps (nbr_sec_mean)
    Daterange = [Daterange[i] for i in range(len(Daterange)) if i%nbr_sec_mean==0]
    print('The starting date of the selected datetime list is: %s, and the ending date is: %s'%(str(Daterange[0]), str(Daterange[-1])))
    
    #######################################################################################################################
    #add new timestamp to the initial file
    df_date = pd.DataFrame({'New_Timestamp':Daterange})
    df_date['New_Timestamp'] = df_date['New_Timestamp'].map(lambda x: pd.to_datetime(x))
    df_ts = pd.merge_asof(df_ts, df_date, left_on=['Timestamp'], right_on=['New_Timestamp'], direction='forward')
    #df_ts['New_Timestamp_old'] = df_ts['Timestamp'].map(lambda x: min([d for d in Daterange if d >= x], default=np.nan))
    
    #aggregate (by using groupby: for each hen take its time serie and find the most frequent zone per new_timestamp)
    for h in tqdm.tqdm([x for x in df_ts.columns if x.startswith('hen_')]):
        df_ = df_ts[[h,'New_Timestamp']].copy()
        df_['nbr_sec'] = nbr_sec
        #df_verification = df_ts.groupby(['New_Timestamp']).agg({'Timestamp':['max', 'min']}).reset_index()
        #df_verification.to_csv(os.path.join(path_ ,id_run+'_ts_MostFrequentZone_period_VERIFICATION'+str(nbr_sec_mean)+'_'+str(mi).split(' ')[0]+'_'+str(ma).split(' ')[0]+'_'+h+'.csv'), sep=';', index=False)
        df__ = df_.groupby(['New_Timestamp',h])['nbr_sec'].sum().reset_index() #sum to count as we have seconds
        df_final = df__.groupby(['New_Timestamp'])[h,'nbr_sec'].agg(lambda x: tuple(x)).reset_index()
        df_final['most_frequent_zone'] = df_final.apply(lambda x: x[h][x['nbr_sec'].index(max(x['nbr_sec']))], axis=1)
        df_final['nbr_duration_per_zone'] = df_final.apply(lambda x: str({x[h][k]:x['nbr_sec'][k] for k in range(len(x[h]))}), axis=1)
        df_final['nbr_lost_duration_per_zone'] = df_final['nbr_duration_per_zone'].map(lambda x: str({z:v for z,v in eval(x).items() if \
                                                                           v!=max(eval(x).values())}))
        df_final['nbr_lost_duration'] = df_final['nbr_lost_duration_per_zone'].map(lambda x: sum(eval(x).values()))
        df_final['perc_lost_duration'] = df_final['nbr_lost_duration'].map(lambda x: x/nbr_sec_mean*100)
        df_final['day'] = df_final['New_Timestamp'].map(lambda x: dt.datetime(x.year,x.month,x.day))
        if save:
            df_final.to_csv(os.path.join(path_ ,id_run+'_ts_MostFrequentZone_period'+str(nbr_sec_mean)+'_'+str(mi).split(' ')[0]+\
                                         '_'+str(ma).split(' ')[0]+'_'+h+'.csv'), sep=';', index=False)
    
    #running time info and return final cleaned df
    END_TIME = time.clock()
    print ("Total running time: %.2f mn" %((END_TIME-START_TIME)/60))
    
    return df_final

In [9]:
df['month'] = df['Timestamp'].map(lambda x: x.month) 
print(df.shape)
df.head(3)

(12453168, 11)


,Timestamp,HenID,Zone,PenID,log_file_name,date,ts_order_logname,ts_order_list,ms,Timestamp_initial,month
0,2019-10-22 07:37:52.000000,hen_71,3 Zone,10,2019-10-16_113343_2019-11-02_060257csv,2019-10-22,2019-10-16_113343_2019-11-02_060257csv_292,2019-10-16_113343_2019-11-02_060257csv_292 201...,0.000000,2019-10-22 07:37:52,10
1,2019-10-22 07:37:52.000000,hen_39,4 Zone,12,2019-10-16_113343_2019-11-02_060257csv,2019-10-22,2019-10-16_113343_2019-11-02_060257csv_293,2019-10-16_113343_2019-11-02_060257csv_293 201...,0.000000,2019-10-22 07:37:52,10
2,2019-10-22 07:37:52.166666,hen_71,3 Zone,10,2019-10-16_113343_2019-11-02_060257csv,2019-10-22,2019-10-16_113343_2019-11-02_060257csv_294,2019-10-16_113343_2019-11-02_060257csv_292 201...,166666.666667,2019-10-22 07:37:52,10


In [5]:
df = df.sort_values(['Timestamp'], ascending=True)
x0 = df.shape[0]
#No need now to have no duplicate timestamp per hen:
#df = df.groupby(['Timestamp','HenID'])[col_ts].agg(lambda x: list(x)[0]).reset_index()  
df_hens = df.pivot(index='Timestamp', columns='HenID', values='Zone')

#fill "None" values with the last non-empty value (by propagating last valHenID observation forward to next valHenID)
#In order to fill in between timestamp, ie. timestamp that another hen had, then the other should also have their latest zone 
#entered instead of nan Note that the first ones will stay None
df_hens = df_hens.fillna(method='ffill')

#Warning: not all hens have same initial/enddate!
#add missing dates 
mi = min(df['Timestamp'].tolist())
ma = max(df['Timestamp'].tolist())
print('The initial starting date in over all is: %s, and the ending date will be: %s'%(str(mi), str(ma)))
print('But note that birds may have different ending and starting date')

The initial starting date in over all is: 2019-10-22 07:37:52, and the ending date will be: 2020-07-18 23:59:59
But note that birds may have different ending and starting date
and after ending the last day at midnight : 2019-10-22 07:37:52, and the ending date will be: 2020-07-18 23:59:59


In [11]:
#add dates until minuit of the last day
ma = dt.datetime(ma.year,ma.month,ma.day,23,59,59)+dt.timedelta(seconds=1)
mi = dt.datetime(mi.year,mi.month,mi.day,0,0,0)
print('and after ending the last day at midnight : %s, and the ending date will be: %s'%(str(mi), str(ma)))
Daterange = pd.date_range(start=mi, end=ma, freq='min') 
print(len(Daterange))
Daterange[0:10]

and after ending the last day at midnight : 2019-10-22 00:00:00, and the ending date will be: 2020-07-21 00:00:00
393121


DatetimeIndex(['2019-10-22 00:00:00', '2019-10-22 00:01:00',
               '2019-10-22 00:02:00', '2019-10-22 00:03:00',
               '2019-10-22 00:04:00', '2019-10-22 00:05:00',
               '2019-10-22 00:06:00', '2019-10-22 00:07:00',
               '2019-10-22 00:08:00', '2019-10-22 00:09:00'],
              dtype='datetime64[ns]', freq='T')

In [ ]:
#add missing seconds (i.e. all seconds that never had a record) and fillnan with last non-nan values by propagating last 
#valHenID observation (even if its an observation that will be removed) forward to next valHenID
df_hens_ = df_hens.reindex(Daterange, method='ffill').reset_index()

In [18]:
df_hens.tail(5)

HenID,hen_1,hen_10,hen_100,hen_101,hen_102,hen_103,hen_104,hen_105,hen_106,hen_107,...,hen_90,hen_91,hen_92,hen_93,hen_94,hen_95,hen_96,hen_97,hen_98,hen_99
Timestamp,,,,,,,,,,,,,,,,,,,,,
2020-07-18 23:59:47,5 Zone,5 Zone,5 Zone,5 Zone,5 Zone,5 Zone,5 Zone,3 Zone,2 Zone,5 Zone,...,5 Zone,5 Zone,3 Zone,5 Zone,5 Zone,2 Zone,5 Zone,3 Zone,2 Zone,5 Zone
2020-07-18 23:59:49,5 Zone,5 Zone,5 Zone,5 Zone,5 Zone,5 Zone,5 Zone,3 Zone,2 Zone,5 Zone,...,5 Zone,5 Zone,3 Zone,5 Zone,5 Zone,2 Zone,5 Zone,3 Zone,2 Zone,5 Zone
2020-07-18 23:59:50,5 Zone,5 Zone,5 Zone,5 Zone,5 Zone,5 Zone,5 Zone,3 Zone,2 Zone,5 Zone,...,5 Zone,5 Zone,3 Zone,5 Zone,5 Zone,2 Zone,5 Zone,3 Zone,2 Zone,5 Zone
2020-07-18 23:59:56,5 Zone,5 Zone,5 Zone,5 Zone,5 Zone,5 Zone,5 Zone,3 Zone,2 Zone,5 Zone,...,5 Zone,5 Zone,3 Zone,5 Zone,5 Zone,2 Zone,5 Zone,3 Zone,2 Zone,5 Zone
2020-07-18 23:59:59,5 Zone,5 Zone,5 Zone,5 Zone,5 Zone,5 Zone,5 Zone,3 Zone,2 Zone,5 Zone,...,5 Zone,5 Zone,3 Zone,5 Zone,5 Zone,2 Zone,5 Zone,3 Zone,2 Zone,5 Zone


In [19]:
df_hens_.tail(5)

HenID,index,hen_1,hen_10,hen_100,hen_101,hen_102,hen_103,hen_104,hen_105,hen_106,...,hen_90,hen_91,hen_92,hen_93,hen_94,hen_95,hen_96,hen_97,hen_98,hen_99
393116,2020-07-20 23:56:00,5 Zone,5 Zone,5 Zone,5 Zone,5 Zone,5 Zone,5 Zone,3 Zone,2 Zone,...,5 Zone,5 Zone,3 Zone,5 Zone,5 Zone,2 Zone,5 Zone,3 Zone,2 Zone,5 Zone
393117,2020-07-20 23:57:00,5 Zone,5 Zone,5 Zone,5 Zone,5 Zone,5 Zone,5 Zone,3 Zone,2 Zone,...,5 Zone,5 Zone,3 Zone,5 Zone,5 Zone,2 Zone,5 Zone,3 Zone,2 Zone,5 Zone
393118,2020-07-20 23:58:00,5 Zone,5 Zone,5 Zone,5 Zone,5 Zone,5 Zone,5 Zone,3 Zone,2 Zone,...,5 Zone,5 Zone,3 Zone,5 Zone,5 Zone,2 Zone,5 Zone,3 Zone,2 Zone,5 Zone
393119,2020-07-20 23:59:00,5 Zone,5 Zone,5 Zone,5 Zone,5 Zone,5 Zone,5 Zone,3 Zone,2 Zone,...,5 Zone,5 Zone,3 Zone,5 Zone,5 Zone,2 Zone,5 Zone,3 Zone,2 Zone,5 Zone
393120,2020-07-21 00:00:00,5 Zone,5 Zone,5 Zone,5 Zone,5 Zone,5 Zone,5 Zone,3 Zone,2 Zone,...,5 Zone,5 Zone,3 Zone,5 Zone,5 Zone,2 Zone,5 Zone,3 Zone,2 Zone,5 Zone


In [16]:
print(df_hens.shape, df_hens_.shape)

(8740161, 112) (393121, 113)


In [21]:
print(df.shape)
df.head(3)

(12453168, 10)


,Timestamp,HenID,Zone,PenID,log_file_name,date,ts_order_logname,ts_order_list,ms,Timestamp_initial
0,2019-10-22 07:37:52.000000,hen_71,3 Zone,10,2019-10-16_113343_2019-11-02_060257csv,2019-10-22,2019-10-16_113343_2019-11-02_060257csv_292,2019-10-16_113343_2019-11-02_060257csv_292 201...,0.000000,2019-10-22 07:37:52
1,2019-10-22 07:37:52.000000,hen_39,4 Zone,12,2019-10-16_113343_2019-11-02_060257csv,2019-10-22,2019-10-16_113343_2019-11-02_060257csv_293,2019-10-16_113343_2019-11-02_060257csv_293 201...,0.000000,2019-10-22 07:37:52
2,2019-10-22 07:37:52.166666,hen_71,3 Zone,10,2019-10-16_113343_2019-11-02_060257csv,2019-10-22,2019-10-16_113343_2019-11-02_060257csv_294,2019-10-16_113343_2019-11-02_060257csv_292 201...,166666.666667,2019-10-22 07:37:52


In [71]:
def duration4bining(y,x):
    #y = next_record_date, x = actual_record_date
    if (pd.isnull(y)) | (pd.isnull(x)):
        return np.datetime64('NaT')
    return min(60-(x-dt.datetime(x.year, x.month, x.day, x.hour, x.minute, 0,0)).total_seconds(),(y-x).total_seconds())
#example 1
#x = dt.datetime(2020,12,2,12,32,2,0)
#y = dt.datetime(2020,12,2,12,33,34,0)
#duration4bining(y,x) #58.0
#example 2
#x = dt.datetime(2020,12,2,12,33,12,0)
#y = dt.datetime(2020,12,2,12,33,34,0)
#duration4bining(y,x) #22.0
#example 3
#x = np.nan
#y = dt.datetime(2020,12,2,12,33,34,0)
#duration4bining(y,x) #numpy.datetime64('NaT')
#example 4
#x = dt.datetime(2020,10,22, 7, 38, 1, 66666)
#y = dt.datetime(2020,10,22, 10, 4, 50, 0)
#duration4bining(y,x) #58.933334

#######################################################################################################################
############################################# Compute duration for bining #############################################
#######################################################################################################################

print_color((('-----------------------------------------------------------------------------------------------','blue'),))
print_color((('Compute duration for bining','blue'),))
print_color((('-----------------------------------------------------------------------------------------------','blue'),))

li_df = []
#more efficient to do it per hen, as it wont need to search in the whole dataframe, and we can simply shift the timestamp column
for i, df_hen in tqdm.tqdm(df.groupby(['HenID'])):
    #as the next record date (sort by date, then simply shift by one row and add nan at then end)
    df_hen = df_hen.sort_values(['Timestamp'], ascending=True) #ts_order
    #same date, one must take the last recorded one & sorting by date might change it. Also it already should be sorted by date
    df_hen['next_record_date'] = df_hen['Timestamp'].tolist()[1:]+[np.nan]
    #compute duration
    df_hen['duration_maximised4bining'] = df_hen.apply(lambda x: duration4bining(x['next_record_date'],x['Timestamp']), axis=1)
    li_df.append(df_hen)
#put again in one dataframe
df_ = pd.concat(li_df)
#dont care about the false positive warning  

100%|████████████████████████████████████████████████████████████████████████████████| 112/112 [13:49<00:00,  7.40s/it]


In [73]:
df_[['HenID','Timestamp','duration_maximised4bining','next_record_date']].head(5)

,HenID,Timestamp,duration_maximised4bining,next_record_date
61,hen_1,2019-10-22 07:38:01.000000,0.333333,2019-10-22 07:38:01.333333
62,hen_1,2019-10-22 07:38:01.333333,0.333333,2019-10-22 07:38:01.666666
64,hen_1,2019-10-22 07:38:01.666666,58.3333,2019-10-22 10:04:50.000000
5739,hen_1,2019-10-22 10:04:50.000000,0.333333,2019-10-22 10:04:50.333333
5741,hen_1,2019-10-22 10:04:50.333333,0.333333,2019-10-22 10:04:50.666666


In [80]:
#######################################################################################################################
##### create a list of dates that we want starting from our initial and end dates with the wanted binning period ######
#######################################################################################################################

#on arondi a la minute du bas/haut pour ne pas rater des records 
mi = min(df_['Timestamp'].tolist())
ma = max(df_['Timestamp'].tolist())
Daterange = pd.date_range(start = mi-dt.timedelta(seconds=mi.second), 
                          end = ma+dt.timedelta(seconds=60-ma.second), 
                          freq = 'min')    
print('The starting date of the datetime list is: %s, and the ending date is: %s'%(str(Daterange[0]), str(Daterange[-1])))
print(len(Daterange))
Daterange[0:10]

The starting date of the datetime list is: 2019-10-22 07:37:00, and the ending date is: 2020-07-19 00:00:00
389784


In [81]:
Daterange[0:10]

DatetimeIndex(['2019-10-22 07:37:00', '2019-10-22 07:38:00',
               '2019-10-22 07:39:00', '2019-10-22 07:40:00',
               '2019-10-22 07:41:00', '2019-10-22 07:42:00',
               '2019-10-22 07:43:00', '2019-10-22 07:44:00',
               '2019-10-22 07:45:00', '2019-10-22 07:46:00'],
              dtype='datetime64[ns]', freq='T')

In [83]:
df_.head(2)

,Timestamp,HenID,Zone,PenID,log_file_name,date,ts_order_logname,ts_order_list,ms,Timestamp_initial,next_record_date,duration_maximised4bining
61,2019-10-22 07:38:01.000000,hen_1,5 Zone,10,2019-10-16_113343_2019-11-02_060257csv,2019-10-22,2019-10-16_113343_2019-11-02_060257csv_313,2019-10-16_113343_2019-11-02_060257csv_313 201...,0.000000,2019-10-22 07:38:01,2019-10-22 07:38:01.333333,0.333333
62,2019-10-22 07:38:01.333333,hen_1,5 Zone,10,2019-10-16_113343_2019-11-20_150252csv,2019-10-22,2019-10-16_113343_2019-11-20_150252csv_313,2019-10-16_113343_2019-11-02_060257csv_313 201...,333333.333333,2019-10-22 07:38:01,2019-10-22 07:38:01.666666,0.333333


In [106]:
#add new timestamp to the initial file
df_date = pd.DataFrame({'New_Timestamp':Daterange})
df_date['New_Timestamp'] = df_date['New_Timestamp'].map(lambda x: pd.to_datetime(x))
df_ = df_.sort_values(['Timestamp'], ascending=True)
df_ts = pd.merge_asof(df_[df_['HenID']=='hen_1'], df_date, 
                      left_on=['Timestamp'], right_on=['New_Timestamp'], direction='forward')
print(df_ts.shape)
df_ts = pd.merge(df_ts, df_date, on=['New_Timestamp'], how='outer')
print(df_ts.shape)
#df_ts['New_Timestamp_old'] = df_ts['Timestamp'].map(lambda x: min([d for d in Daterange if d >= x], default=np.nan))
df_ts = df_ts.sort_values(['New_Timestamp'], ascending=True)
df_ts[['Timestamp','Zone','duration_maximised4bining','next_record_date',
                                'New_Timestamp']].head(10)

(109510, 13)
(468069, 13)


,HenID,Timestamp,Zone,duration_maximised4bining,next_record_date,New_Timestamp
109510,NaN,NaT,NaN,NaN,NaT,2019-10-22 07:37:00
109511,NaN,NaT,NaN,NaN,NaT,2019-10-22 07:38:00
0,hen_1,2019-10-22 07:38:01.000000,5 Zone,0.333333,2019-10-22 07:38:01.333333,2019-10-22 07:39:00
2,hen_1,2019-10-22 07:38:01.666666,5 Zone,58.3333,2019-10-22 10:04:50.000000,2019-10-22 07:39:00
1,hen_1,2019-10-22 07:38:01.333333,5 Zone,0.333333,2019-10-22 07:38:01.666666,2019-10-22 07:39:00
109512,NaN,NaT,NaN,NaN,NaT,2019-10-22 07:40:00
109513,NaN,NaT,NaN,NaN,NaT,2019-10-22 07:41:00
109514,NaN,NaT,NaN,NaN,NaT,2019-10-22 07:42:00
109515,NaN,NaT,NaN,NaN,NaT,2019-10-22 07:43:00
109516,NaN,NaT,NaN,NaN,NaT,2019-10-22 07:44:00


In [ ]:
#add new timestamp to the initial file
df_date = pd.DataFrame({'New_Timestamp':Daterange})
df_date['New_Timestamp'] = df_date['New_Timestamp'].map(lambda x: pd.to_datetime(x))
df_ = df_.sort_values(['Timestamp'], ascending=True)
df_ts = pd.merge_asof(df_, df_date, left_on=['Timestamp'], right_on=['New_Timestamp'], direction='forward')
#df_ts['New_Timestamp_old'] = df_ts['Timestamp'].map(lambda x: min([d for d in Daterange if d >= x], default=np.nan))
df_ts[df_ts['HenID']=='hen_1'][['HenID','Timestamp','Zone','duration_maximised4bining','next_record_date',
                                'New_Timestamp']].head(10)

In [99]:
df_hens = df_ts[df_ts['HenID']=='hen_1'].pivot(index='Timestamp', columns='HenID', values='Zone')
print(df_hens.shape)
df_hens.head(5)

(109510, 1)


HenID,hen_1
Timestamp,
2019-10-22 07:38:01.000000,5 Zone
2019-10-22 07:38:01.333333,5 Zone
2019-10-22 07:38:01.666666,5 Zone
2019-10-22 10:04:50.000000,2 Zone
2019-10-22 10:04:50.333333,2 Zone


In [90]:
df_hens = df.pivot(index='Timestamp', columns='HenID', values=col_ts)

#fill "None" values with the last non-empty value (by propagating last valHenID observation forward to next valHenID)
#In order to fill in between timestamp, ie. timestamp that another hen had, then the other should also have their latest zone 
#entered instead of nan Note that the first ones will stay None
df_hens = df_hens.fillna(method='ffill')
df_ts = df_ts.reindex(Daterange, method='ffill').reset_index()

TypeError: '<' not supported between instances of 'Timestamp' and 'int'

In [ ]:
df_ts[df_ts['HenID']=='hen_1'][['HenID','Timestamp','duration_maximised4bining','next_record_date','New_Timestamp']].head(10)

#### turn all bining hens time series into the usual ts csv file (one column per hen)

In [ ]:
#for path_ in glob.glob(os.path.join(path_extracted_data, 'HensTimeSeries', id_run+'_ts_MostFrequentZone_period60_?_?_hen_*.csv')):
li_df = []
for path_ in tqdm.tqdm(glob.glob(os.path.join(path_extracted_data, 'HensTimeSeries', 
                                    id_run+'_ts_MostFrequentZone_period60_2019-11-01_2020-03-26_hen_*.csv'))):
    name_ = path_.split('\\')[-1].split('_MostFrequentZone_')[-1].split('.csv')[0]
    HenID = 'hen_'+name_.split('hen_')[-1]
    df_h = pd.read_csv(path_, sep=';', parse_dates=['New_Timestamp']) 
    df_h = df_h.filter(['New_Timestamp','most_frequent_zone'],axis=1).reset_index(drop=True)
    df_h.rename(columns={'most_frequent_zone':HenID}, inplace=True)
    li_df.append(df_h)

df = pd.DataFrame(columns=['New_Timestamp'])
for df_h in li_df:
    df = pd.merge(df, df_h, on='New_Timestamp', how='outer')
df.to_csv(os.path.join(path_extracted_data,id_run+'ts_MostFrequentZone_period60_2019-11-01_2020-03-26_allhens.csv'), sep=';', index=False)
print(df.shape)
df.head(3)

In [ ]:
df.tail(3)

#VACCINATION 2
for minute in [0.5,1,2,3,4,5,10,30]:
    print('Lets compute with time interval ', minute)
    simple_cleaning_experiment2(df_ts, config, 
                                nbr_sec_mean=int(minute*60),
                                mi=dt.datetime(2019,12,20,0,0,0), 
                                ma=dt.datetime(2019,12,25,23,59,59))

# Looking at lost info depending on zones when bining

In [ ]:
#this script is temporaire
li_path = glob.glob(os.path.join(path_extracted_data, 'HensTimeSeries', id_run+'_ts_MostFrequentZone_period*.csv'))

dico_mi_li_df = {}
for path_ in tqdm.tqdm(li_path):
    h = 'hen_'+path_.split('hen_')[1][:-4]
    if 'VERIFICATION' not in path_:
        period = int(path_.split('period')[1].split('_')[0])
        df_ = pd.read_csv(path_, sep=';', parse_dates=['New_Timestamp'])
        d1 = path_.split('period')[1].split('_')[1]
        d2 = path_.split('period')[1].split('_')[2]
        mi=dt.datetime(int(d1[0:4]),int(d1[5:7]),int(d1[8:]),0,0,0)
        ma=dt.datetime(int(d2[0:4]),int(d2[5:7]),int(d2[8:]),23,59,59)
        n = str(mi)+'   '+str(ma)
        df_['HenID'] = h
        #if 'nbr_lost_duration_per_zone' in df_.columns:
        df_['nbr_lost_duration_per_zone'] = df_['nbr_lost_duration_per_zone'].map(lambda x: eval(x))
        dico_ = dict(functools.reduce(operator.add, map(collections.Counter, df_['nbr_lost_duration_per_zone'].tolist()))) 
        df_['nbr_duration_per_zone'] = df_['nbr_duration_per_zone'].map(lambda x: eval(x))
        dico_all = dict(functools.reduce(operator.add, map(collections.Counter, df_['nbr_duration_per_zone'].tolist()))) 
        if n not in dico_mi_li_df:
            dico_mi_li_df[n] = []
        for z in dico_all.keys():
            dico_mi_li_df[n].append({'period':period, 'HenID':h, 
                                     'percentage_lost_duration':dico_.get(z,0)/dico_all[z]*100, 'zone': z})

#plot
for n, li_df in dico_mi_li_df.items():
    df_plot = pd.DataFrame(li_df)
    #plot for each datarange a plot showing the evolution of duration lost per zone and period
    fig, ax = plt.subplots(figsize=(12,6))
    ax = sns.boxplot(x="period", y="percentage_lost_duration", hue="zone", data=df_plot) #, palette="Set3")
    plt.title(n)
    plt.savefig(os.path.join(path_extracted_data,'visual','for_verification', 
                             id_run+'_percentage_lost_duration_during_bining'+n.split(':')[0][:-3]+'_'+n.split(':')[2][5:-3]+'.png'), dpi=300,
                             format='png', bbox_inches='tight')
    plt.show()

#-->zone 1: si flickering pas possible peut-être que nous devrons le garder en plus deu bining, puis choisir un bining 
#plus petit ou égale à 5mn peut etre :) 